In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Set seed for reproducibility
np.random.seed(42)

# Generate synthetic data
num_rows = 300

# Generate Transaction IDs and Customer IDs
transaction_ids = np.arange(1001, 1001 + num_rows)
customer_ids = np.random.randint(2001, 2500, num_rows)

# Generate Customer Names with more missing values
customer_names = np.random.choice(["Alice", "Bob", "Charlie", "David", "Emma", None], num_rows,
                                  p=[0.15, 0.15, 0.15, 0.15, 0.1, 0.3])  # More missing values

# Generate Purchase Amounts with outliers and text values
purchase_amounts = np.random.uniform(5, 500, num_rows).astype(str)  # Stored as strings initially
purchase_amounts[random.randint(0, num_rows-1)] = None  # Add missing value
purchase_amounts[random.randint(0, num_rows-1)] = "Ten Dollars"  # Add non-numeric value
purchase_amounts[random.randint(0, num_rows-1)] = "10000"  # Add extreme outlier

# Generate Transaction Dates with more inconsistencies
transaction_dates = [(datetime.today() - timedelta(days=random.randint(0, 365))).strftime(
    random.choice(["%Y-%m-%d", "%d/%m/%Y", "%m-%d-%Y", "%B %d, %Y"])) for _ in range(num_rows)]
transaction_dates[random.randint(0, num_rows-1)] = None  # Add missing value
transaction_dates[random.randint(0, num_rows-1)] = "January 32, 2023"  # Add incorrect date
transaction_dates[random.randint(0, num_rows-1)] = "2027-12-25"  # Add future date

# Generate Product Categories with typos
product_categories = np.random.choice(["Electronics", "Clothing", "Books", "Home & Kitchen", "Beauty",
                                       "Elctronics", "Clothng", "Hme & Kitchen"], num_rows)  # Some typos

# Generate Payment Methods with inconsistencies
payment_methods = np.random.choice(["Credit Card", "credit_card", "Paypal", "PayPal", "BANK_TRANSFER",
                                    "bank transfer", "banktransfer", "paypal "], num_rows)  # Some extra spaces and variations

# Introduce more duplicate transactions
duplicates = 10  # Number of duplicate rows to add
duplicate_indices = np.random.choice(num_rows, duplicates, replace=False)

# Create DataFrame
data = {
    "Transaction_ID": transaction_ids.tolist(),
    "Customer_ID": customer_ids.tolist(),
    "Customer Name": customer_names.tolist(),
    "Purchase Amount ($)": purchase_amounts,
    "Transaction Date": transaction_dates,
    "Product Category": product_categories.tolist(),
    "Payment Method": payment_methods.tolist()
}

df_messy = pd.DataFrame(data)

# Add duplicate rows
df_messy = pd.concat([df_messy, df_messy.iloc[duplicate_indices]], ignore_index=True)

# Save dataset
df_messy.to_csv("messy_transaction_data_v2.csv", index=False)

print("Dataset saved as 'messy_transaction_data_v2.csv'")


Dataset saved as 'messy_transaction_data_v2.csv'


In [2]:
df = pd.read_csv("messy_transaction_data_v2.csv")
df.head()
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Transaction_ID       310 non-null    int64 
 1   Customer_ID          310 non-null    int64 
 2   Customer Name        218 non-null    object
 3   Purchase Amount ($)  309 non-null    object
 4   Transaction Date     309 non-null    object
 5   Product Category     310 non-null    object
 6   Payment Method       310 non-null    object
dtypes: int64(2), object(5)
memory usage: 17.1+ KB


In [3]:
df.columns

Index(['Transaction_ID', 'Customer_ID', 'Customer Name', 'Purchase Amount ($)',
       'Transaction Date', 'Product Category', 'Payment Method'],
      dtype='object')

# 1️⃣ Handling Missing Values

In [4]:
# Check for missing values in the dataset.
df.isnull().sum()

,0
Transaction_ID,0
Customer_ID,0
Customer Name,92
Purchase Amount ($),1
Transaction Date,1
Product Category,0
Payment Method,0


In [5]:
# Fill missing values in Customer Name with "Unknown".
df["Customer Name"].fillna("Unknown", inplace=True)
df.isnull().sum()

<ipython-input-5-b8d44bdef80a>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Customer Name"].fillna("Unknown", inplace=True)


,0
Transaction_ID,0
Customer_ID,0
Customer Name,0
Purchase Amount ($),1
Transaction Date,1
Product Category,0
Payment Method,0


In [6]:
def to_snake_case(column_name):
    snake_chars = []
    prev_underscore = False  # Track if previous character was an underscore

    for char in column_name:
        lower_char = char.lower()

        # Keep letters, numbers, and existing underscores
        if lower_char.isalnum() or lower_char == '_':
            if lower_char == '_':
                # Add underscore only if previous wasn't an underscore
                if not prev_underscore:
                    snake_chars.append('_')
                    prev_underscore = True
            else:
                snake_chars.append(lower_char)
                prev_underscore = False
        # Replace all other characters with underscores
        else:
            if not prev_underscore:
                snake_chars.append('_')
                prev_underscore = True

    # Join characters and clean up edges
    result = ''.join(snake_chars).strip('_')
    # Handle empty strings from names with only special characters
    return result if result else '_'

# Apply the conversion to all column names
df.columns = [to_snake_case(col) for col in df.columns]
df.columns

Index(['transaction_id', 'customer_id', 'customer_name', 'purchase_amount',
       'transaction_date', 'product_category', 'payment_method'],
      dtype='object')

In [11]:
# Drop rows where Purchase Amount ($) or Transaction Date is missing.
cleaned_df = df.dropna(subset = ["purchase_amount", "transaction_date"])
cleaned_df.isnull().sum()

,0
transaction_id,0
customer_id,0
customer_name,0
purchase_amount,0
transaction_date,0
product_category,0
payment_method,0


# 2️⃣ Fixing Data Types

In [33]:
from sre_constants import error
# Convert Purchase Amount ($) to a float.
cleaned_df['purchase_amount'].unique()
cleaned_df['purchase_amount'] = cleaned_df['purchase_amount'].replace({'Ten Dollars': '10', '10000': '1000'}, regex=True)
cleaned_df['purchase_amount'] = cleaned_df['purchase_amount'].astype(float)

# Convert Transaction Date to a proper datetime format.
cleaned_df['transaction_date'] = pd.to_datetime(cleaned_df['transaction_date'], errors = "coerce")
cleaned_df.dropna(subset = ["transaction_date"], inplace=True)
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 0 to 304
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    39 non-null     int64         
 1   customer_id       39 non-null     int64         
 2   customer_name     39 non-null     object        
 3   purchase_amount   39 non-null     float64       
 4   transaction_date  39 non-null     datetime64[ns]
 5   product_category  39 non-null     object        
 6   payment_method    39 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 2.4+ KB


<ipython-input-33-a237b1c9b782>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['purchase_amount'] = cleaned_df['purchase_amount'].replace({'Ten Dollars': '10', '10000': '1000'}, regex=True)
<ipython-input-33-a237b1c9b782>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['purchase_amount'] = cleaned_df['purchase_amount'].astype(float)
<ipython-input-33-a237b1c9b782>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [32]:
cleaned_df['payment_method'].unique()
change_words = {
    'credit_card': 'credit card', 'bank_transfer': 'bank transfer',
    'banktransfer': 'bank transfer', 'paypal ': 'paypal',
    'bank ransfer':'bank transfer'}
cleaned_df['payment_method'] = cleaned_df['payment_method'].replace(change_words)
cleaned_df['payment_method'].unique()

<ipython-input-32-2d666d915cc5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['payment_method'] = cleaned_df['payment_method'].replace(change_words)


array(['paypal', 'credit card', 'bank transfer'], dtype=object)

In [34]:
# Extract the year and month from Transaction Date into new columns.
cleaned_df['transaction_year'] = cleaned_df['transaction_date'].dt.year
cleaned_df['transaction_month'] = cleaned_df['transaction_date'].dt.month_name()
cleaned_df.head()

<ipython-input-34-58fba781529b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['transaction_year'] = cleaned_df['transaction_date'].dt.year
<ipython-input-34-58fba781529b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['transaction_month'] = cleaned_df['transaction_date'].dt.month_name()


,transaction_id,customer_id,customer_name,purchase_amount,transaction_date,product_category,payment_method,transaction_year,transaction_month
0,1001,2103,Charlie,287.879147,2024-07-09,Elctronics,paypal,2024,July
1,1002,2436,Bob,371.782061,2025-11-02,Clothng,credit card,2025,November
10,1011,2467,Emma,266.543210,2024-06-08,Books,bank transfer,2024,June
24,1025,2492,Bob,150.774504,2024-09-05,Beauty,credit card,2024,September
36,1037,2236,Bob,325.189939,2024-09-12,Beauty,credit card,2024,September


In [36]:
cleaned_df['product_category'].unique()
changed_categories = {'Elctronics': 'Electronics',
                      'Clothng': 'Clothing',
                      'Hme & Kitchen': 'Home & Kitchen'}
cleaned_df['product_category'] = cleaned_df['product_category'].replace(changed_categories)
cleaned_df['product_category'].unique()

<ipython-input-36-9caa106993dd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['product_category'] = cleaned_df['product_category'].replace(changed_categories)


array(['Electronics', 'Clothing', 'Books', 'Beauty', 'Home & Kitchen'],
      dtype=object)

In [45]:
# Check for duplicate rows in the dataset.
duplicate_rows = cleaned_df[cleaned_df.duplicated(keep=False)]
cleaned_df.drop_duplicates(inplace =True)
cleaned_df.reset_index(drop=True, inplace=True)
cleaned_df.shape

<ipython-input-45-ce9a2c0d8a2a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.drop_duplicates(inplace =True)


(38, 9)

In [46]:
cleaned_df

,transaction_id,customer_id,customer_name,purchase_amount,transaction_date,product_category,payment_method,transaction_year,transaction_month
0,1001,2103,Charlie,287.879147,2024-07-09,Electronics,paypal,2024,July
1,1002,2436,Bob,371.782061,2025-11-02,Clothing,credit card,2025,November
2,1011,2467,Emma,266.543210,2024-06-08,Books,bank transfer,2024,June
3,1025,2492,Bob,150.774504,2024-09-05,Beauty,credit card,2024,September
4,1037,2236,Bob,325.189939,2024-09-12,Beauty,credit card,2024,September
5,1038,2345,Bob,197.192850,2024-11-09,Clothing,bank transfer,2024,November
6,1039,2049,Unknown,118.550399,2024-11-05,Home & Kitchen,credit card,2024,November
7,1054,2244,David,459.992393,2024-09-07,Books,bank transfer,2024,September
8,1059,2135,David,208.571283,2024-02-12,Electronics,bank transfer,2024,February
9,1063,2274,Unknown,191.485790,2024-04-11,Clothing,credit card,2024,April


# 5️⃣ Dealing with Outliers

In [52]:
Q1 = cleaned_df['purchase_amount'].quantile(0.25)
Q3 = cleaned_df['purchase_amount'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = cleaned_df[
    (cleaned_df['purchase_amount'] < lower_bound) |
     (cleaned_df['purchase_amount'] > upper_bound)
     ]
len(outliers)

0